# Step 1: Load All Files

In [1]:
import zipfile
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import statsmodels.api as sm
print(sm.__version__)


# Set the aesthetic style of the plots
sns.set_style("whitegrid")

# Specify the path to the downloaded file and the destination directory
zip_file_path = r'C:\Users\khaie\Desktop\2022 Russia Ukraine War\2022-ukraine-russian-war.zip'
destination_directory = 'path/to/ukraine_russian_war_data/'

# Unzip the file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(destination_directory)

print("Files unzipped successfully")


0.14.4
Files unzipped successfully


### Loading the 3 files

In [2]:
# Load the personnel losses data
personnel_data = pd.read_csv(r'C:\Users\khaie\Desktop\2022 Russia Ukraine War\russia_losses_personnel.csv')

# Load the equipment losses data
equipment_data = pd.read_csv(r'C:\Users\khaie\Desktop\2022 Russia Ukraine War\russia_losses_equipment.csv')

# Load the equipment correction data
equipment_correction_data = pd.read_csv(r'C:\Users\khaie\Desktop\2022 Russia Ukraine War\russia_losses_equipment_correction.csv')

# Display the first few records of each DataFrame
print("Personnel Data")
print(personnel_data.head())
print("\nEquipment Data")
print(equipment_data.head())
print("\nEquipment Correction Data")
print(equipment_correction_data.head())

Personnel Data
         date   day  personnel personnel*  POW
0  2025-02-16  1089     858390      about  NaN
1  2025-02-15  1088     856660      about  NaN
2  2025-02-14  1087     855480      about  NaN
3  2025-02-13  1086     854280      about  NaN
4  2025-02-12  1085     853030      about  NaN

Equipment Data
         date   day  aircraft  helicopter   tank    APC  field artillery  \
0  2025-02-16  1089       370         331  10073  21011            23185   
1  2025-02-15  1088       370         331  10068  20927            23174   
2  2025-02-14  1087       370         331  10057  20910            23115   
3  2025-02-13  1086       370         331  10040  20894            23034   
4  2025-02-12  1085       370         331  10023  20871            22976   

    MRL  military auto  fuel tank  drone  naval ship  anti-aircraft warfare  \
0  1283            NaN        NaN  25377          28                   1067   
1  1283            NaN        NaN  25341          28                   1

# Step 2: Initial Inspection

### 2.1 Personnel Data

In [3]:
# Display basic information about each dataset
print(personnel_data.info())
print(equipment_data.info())
print(equipment_correction_data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1088 entries, 0 to 1087
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   date        1088 non-null   object 
 1   day         1088 non-null   int64  
 2   personnel   1088 non-null   int64  
 3   personnel*  1088 non-null   object 
 4   POW         62 non-null     float64
dtypes: float64(1), int64(2), object(2)
memory usage: 42.6+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1088 entries, 0 to 1087
Data columns (total 19 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   date                       1088 non-null   object 
 1   day                        1088 non-null   int64  
 2   aircraft                   1088 non-null   int64  
 3   helicopter                 1088 non-null   int64  
 4   tank                       1088 non-null   int64  
 5   APC                        1088 non

### 2.2 checking for missing values

In [4]:
# Check for missing values in each dataset
print(personnel_data.isnull().sum())
print(equipment_data.isnull().sum())
print(equipment_correction_data.isnull().sum())

date             0
day              0
personnel        0
personnel*       0
POW           1026
dtype: int64
date                            0
day                             0
aircraft                        0
helicopter                      0
tank                            0
APC                             0
field artillery                 0
MRL                             0
military auto                1023
fuel tank                    1023
drone                           0
naval ship                      0
anti-aircraft warfare           0
special equipment              19
mobile SRBM system           1052
greatest losses direction     885
vehicles and fuel tanks        65
cruise missiles                65
submarines                    566
dtype: int64
date                       0
day                        0
aircraft                   0
helicopter                 0
tank                       0
APC                        0
field artillery            0
MRL                        0
d

### 2.3 Statistics Summary

In [5]:
# Display summary statistics for each dataset
print(personnel_data.describe())
print(equipment_data.describe())
print(equipment_correction_data.describe())


               day      personnel         POW
count  1088.000000    1088.000000   62.000000
mean    545.500000  311275.923713  386.387097
std     314.222851  245252.298974  131.440363
min       2.000000    2800.000000    0.000000
25%     273.750000   85642.500000  389.000000
50%     545.500000  258580.000000  421.000000
75%     817.250000  494035.000000  474.500000
max    1089.000000  858390.000000  496.000000
               day     aircraft   helicopter          tank           APC  \
count  1088.000000  1088.000000  1088.000000   1088.000000   1088.000000   
mean    545.500000   301.499081   281.008272   5014.780331   9932.453125   
std     314.222851    67.468334    64.671322   2818.251147   5505.397746   
min       2.000000    10.000000     7.000000     80.000000    516.000000   
25%     273.750000   278.000000   261.000000   2897.750000   5835.750000   
50%     545.500000   315.000000   316.000000   4367.500000   8482.000000   
75%     817.250000   354.000000   326.000000   7593.75

# Step 3: Clean and Merge Data

### 3.1 Personnel Data Cleanup

In [6]:
# Convert the 'date' column to datetime
personnel_data['date'] = pd.to_datetime(personnel_data['date'])

# Ensure numeric columns are of the correct type
numeric_columns = ['personnel']
for column in numeric_columns:
    personnel_data[column] = pd.to_numeric(personnel_data[column], errors='coerce')

# Handle missing values
personnel_data.fillna(0, inplace=True)

# Sort the data by date
personnel_data.sort_values(by='date', inplace=True)

# Calculate daily personnel changes
personnel_data['daily_personnel_change'] = personnel_data['personnel'].diff().fillna(personnel_data['personnel']).astype(int)

# Display the updated DataFrame
print(personnel_data.head())


           date  day  personnel personnel*    POW  daily_personnel_change
1087 2022-02-25    2       2800      about    0.0                    2800
1086 2022-02-26    3       4300      about    0.0                    1500
1085 2022-02-27    4       4500      about    0.0                     200
1084 2022-02-28    5       5300      about    0.0                     800
1083 2022-03-01    6       5710      about  200.0                     410


### 3.2 Equipment Data Cleanup

In [7]:
# Convert the 'date' column to datetime
equipment_data['date'] = pd.to_datetime(equipment_data['date'])

# Ensure numeric columns are of the correct type
numeric_columns = ['aircraft', 'helicopter', 'tank', 'APC', 'field artillery', 'MRL', 'military auto', 'fuel tank']
for column in numeric_columns:
    # Convert column to string type before using .str accessor
    equipment_data[column] = equipment_data[column].astype(str).str.replace(',', '')
    equipment_data[column] = pd.to_numeric(equipment_data[column], errors='coerce')

# Handle missing values
equipment_data.fillna(0, inplace=True)

# Sort the data by date
equipment_data.sort_values(by='date', inplace=True)

# Calculate daily changes for each type of equipment
for column in numeric_columns:
    equipment_data[f'daily_{column}_change'] = equipment_data[column].diff().fillna(equipment_data[column]).astype(int)

# Display the updated DataFrame
print(equipment_data.head())


           date  day  aircraft  helicopter  tank  APC  field artillery  MRL  \
1087 2022-02-25    2        10           7    80  516               49    4   
1086 2022-02-26    3        27          26   146  706               49    4   
1085 2022-02-27    4        27          26   150  706               50    4   
1084 2022-02-28    5        29          29   150  816               74   21   
1083 2022-03-01    6        29          29   198  846               77   24   

      military auto  fuel tank  ...  cruise missiles  submarines  \
1087          100.0       60.0  ...              0.0         0.0   
1086          130.0       60.0  ...              0.0         0.0   
1085          130.0       60.0  ...              0.0         0.0   
1084          291.0       60.0  ...              0.0         0.0   
1083          305.0       60.0  ...              0.0         0.0   

      daily_aircraft_change  daily_helicopter_change  daily_tank_change  \
1087                     10              

### 3.3 Merging Equipment Correction Data

In [8]:
# Apply corrections from the equipment correction data
equipment_data.update(equipment_correction_data)

# Verify the corrections
print(equipment_data.head())


           date  day  aircraft  helicopter  tank  APC  field artillery  MRL  \
1087 2022-02-25    2        10           7    80  516               49    4   
1086 2022-02-26    3        27          26   146  706               49    4   
1085 2022-02-27    4        27          26   150  706               50    4   
1084 2022-02-28    5        29          29   150  816               74   21   
1083 2022-03-01    6        29          29   198  846               77   24   

      military auto  fuel tank  ...  cruise missiles  submarines  \
1087          100.0       60.0  ...              0.0         0.0   
1086          130.0       60.0  ...              0.0         0.0   
1085          130.0       60.0  ...              0.0         0.0   
1084          291.0       60.0  ...              0.0         0.0   
1083          305.0       60.0  ...              0.0         0.0   

      daily_aircraft_change  daily_helicopter_change  daily_tank_change  \
1087                     10              

# 4. Numbers of Columns for Pers. & Equips. 32

### 4.1 Numbers of Columns

In [9]:
# Number of columns in personnel_data
num_columns_personnel = len(personnel_data.columns)
print(f"Number of columns in personnel_data: {num_columns_personnel}")

# Number of columns in equipment_data
num_columns_equipment = len(equipment_data.columns)
print(f"Number of columns in equipment_data: {num_columns_equipment}")

Number of columns in personnel_data: 6
Number of columns in equipment_data: 27


### 4.2 List of Colunms

In [10]:
from IPython.display import display, HTML

# Create a DataFrame for personnel columns
personnel_columns_df = pd.DataFrame(personnel_data.columns, columns=['Personnel Data Columns'])

# Create a DataFrame for equipment columns
equipment_columns_df = pd.DataFrame(equipment_data.columns, columns=['Equipment Data Columns'])

# Display the DataFrames side by side
display(HTML(personnel_columns_df.to_html(index=False)))
display(HTML(equipment_columns_df.to_html(index=False)))

Personnel Data Columns
date
day
personnel
personnel*
POW
daily_personnel_change


Equipment Data Columns
date
day
aircraft
helicopter
tank
APC
field artillery
MRL
military auto
fuel tank


# 5. Merge All DataFrames

### 5.1 The merge

In [11]:
# Ensure the 'date' column is of datetime type in both DataFrames
personnel_data['date'] = pd.to_datetime(personnel_data['date'])
equipment_data['date'] = pd.to_datetime(equipment_data['date'])

# Merge personnel and equipment data
merged_data = pd.merge(personnel_data, equipment_data, on='date', how='outer')

# Display the merged DataFrame
print(merged_data.head())


        date  day_x  personnel personnel*    POW  daily_personnel_change  \
0 2022-02-25      2       2800      about    0.0                    2800   
1 2022-02-26      3       4300      about    0.0                    1500   
2 2022-02-27      4       4500      about    0.0                     200   
3 2022-02-28      5       5300      about    0.0                     800   
4 2022-03-01      6       5710      about  200.0                     410   

   day_y  aircraft  helicopter   tank  ...  cruise missiles  submarines  \
0    2.0      10.0         7.0   80.0  ...              0.0         0.0   
1    3.0      27.0        26.0  146.0  ...              0.0         0.0   
2    4.0      27.0        26.0  150.0  ...              0.0         0.0   
3    5.0      29.0        29.0  150.0  ...              0.0         0.0   
4    6.0      29.0        29.0  198.0  ...              0.0         0.0   

   daily_aircraft_change  daily_helicopter_change  daily_tank_change  \
0                   

### 5.2 Total Nr.s of Columns after Merge

In [12]:
total_columns_after_merge = len(merged_data.columns)
print(f"Total number of columns after merge: {total_columns_after_merge}")

Total number of columns after merge: 32


### 5.3 List of Merged columns (32)

In [13]:
# Generate more informative columns
merged_data['week'] = merged_data['date'].dt.isocalendar().week
merged_data['month'] = merged_data['date'].dt.month
merged_data['year'] = merged_data['date'].dt.year
merged_data['day_of_week'] = merged_data['date'].dt.day_name()

# Calculate cumulative sums and rolling averages for daily changes
for column in numeric_columns:
    merged_data[f'cumulative_{column}_change'] = merged_data[f'daily_{column}_change'].cumsum()
    merged_data[f'7_day_rolling_avg_{column}_change'] = merged_data[f'daily_{column}_change'].rolling(window=7).mean()
    merged_data[f'30_day_rolling_avg_{column}_change'] = merged_data[f'daily_{column}_change'].rolling(window=30).mean()

# Display the updated DataFrame
print(merged_data.head())

# New columns generated:
# 1. 'week' - ISO calendar week number
# 2. 'month' - Month number
# 3. 'year' - Year number
# 4. 'day_of_week' - Day name of the week
# 5. 'cumulative_<column>_change' - Cumulative sum of daily changes for each numeric column
# 6. '7_day_rolling_avg_<column>_change' - 7-day rolling average of daily changes
# 7. '30_day_rolling_avg_<column>_change' - 30-day rolling average of daily changes

        date  day_x  personnel personnel*    POW  daily_personnel_change  \
0 2022-02-25      2       2800      about    0.0                    2800   
1 2022-02-26      3       4300      about    0.0                    1500   
2 2022-02-27      4       4500      about    0.0                     200   
3 2022-02-28      5       5300      about    0.0                     800   
4 2022-03-01      6       5710      about  200.0                     410   

   day_y  aircraft  helicopter   tank  ...  \
0    2.0      10.0         7.0   80.0  ...   
1    3.0      27.0        26.0  146.0  ...   
2    4.0      27.0        26.0  150.0  ...   
3    5.0      29.0        29.0  150.0  ...   
4    6.0      29.0        29.0  198.0  ...   

   30_day_rolling_avg_field artillery_change  cumulative_MRL_change  \
0                                        NaN                    4.0   
1                                        NaN                    4.0   
2                                        NaN         

# 6. Step 5: Generate Additional Columns

### 6.1 Generating Additional columns

In [14]:
# Generate more informative columns
merged_data['week'] = merged_data['date'].dt.isocalendar().week
merged_data['month'] = merged_data['date'].dt.month
merged_data['year'] = merged_data['date'].dt.year
merged_data['day_of_week'] = merged_data['date'].dt.day_name()

# Calculate cumulative sums and rolling averages for daily changes
for column in numeric_columns:
    merged_data[f'cumulative_{column}_change'] = merged_data[f'daily_{column}_change'].cumsum()
    merged_data[f'7_day_rolling_avg_{column}_change'] = merged_data[f'daily_{column}_change'].rolling(window=7).mean()
    merged_data[f'30_day_rolling_avg_{column}_change'] = merged_data[f'daily_{column}_change'].rolling(window=30).mean()

# Display the updated DataFrame
print(merged_data.head())


        date  day_x  personnel personnel*    POW  daily_personnel_change  \
0 2022-02-25      2       2800      about    0.0                    2800   
1 2022-02-26      3       4300      about    0.0                    1500   
2 2022-02-27      4       4500      about    0.0                     200   
3 2022-02-28      5       5300      about    0.0                     800   
4 2022-03-01      6       5710      about  200.0                     410   

   day_y  aircraft  helicopter   tank  ...  \
0    2.0      10.0         7.0   80.0  ...   
1    3.0      27.0        26.0  146.0  ...   
2    4.0      27.0        26.0  150.0  ...   
3    5.0      29.0        29.0  150.0  ...   
4    6.0      29.0        29.0  198.0  ...   

   30_day_rolling_avg_field artillery_change  cumulative_MRL_change  \
0                                        NaN                    4.0   
1                                        NaN                    4.0   
2                                        NaN         

### 6.2 Total of all merged columns (60) 

In [15]:
total_columns_all = len(merged_data.columns)
print(f"Total number of all columns: {total_columns_all}")

Total number of all columns: 60


### 6.3 List of Merged columns (1-60)

In [16]:
# Create a DataFrame for the columns with their respective numbers
columns_df = pd.DataFrame({
    'Column Number': range(1, len(merged_data.columns) + 1),
    'Column Name': merged_data.columns
})

# Display the DataFrame
columns_df

,Column Number,Column Name
0,1,date
1,2,day_x
2,3,personnel
3,4,personnel*
4,5,POW
5,6,daily_personnel_change
6,7,day_y
7,8,aircraft
8,9,helicopter
9,10,tank


In [17]:
# Generate more informative columns
merged_data['week'] = merged_data['date'].dt.isocalendar().week
merged_data['month'] = merged_data['date'].dt.month
merged_data['year'] = merged_data['date'].dt.year
merged_data['day_of_week'] = merged_data['date'].dt.day_name()

# Calculate cumulative sums and rolling averages for daily changes
for column in numeric_columns:
    merged_data[f'cumulative_{column}_change'] = merged_data[f'daily_{column}_change'].cumsum()
    merged_data[f'7_day_rolling_avg_{column}_change'] = merged_data[f'daily_{column}_change'].rolling(window=7).mean()
    merged_data[f'30_day_rolling_avg_{column}_change'] = merged_data[f'daily_{column}_change'].rolling(window=30).mean()

# Display the updated DataFrame
print(merged_data.head())

# New columns generated:
# 1. 'week' - ISO calendar week number
# 2. 'month' - Month number
# 3. 'year' - Year number
# 4. 'day_of_week' - Day name of the week
# 5. 'cumulative_<column>_change' - Cumulative sum of daily changes for each numeric column
# 6. '7_day_rolling_avg_<column>_change' - 7-day rolling average of daily changes
# 7. '30_day_rolling_avg_<column>_change' - 30-day rolling average of daily changes

        date  day_x  personnel personnel*    POW  daily_personnel_change  \
0 2022-02-25      2       2800      about    0.0                    2800   
1 2022-02-26      3       4300      about    0.0                    1500   
2 2022-02-27      4       4500      about    0.0                     200   
3 2022-02-28      5       5300      about    0.0                     800   
4 2022-03-01      6       5710      about  200.0                     410   

   day_y  aircraft  helicopter   tank  ...  \
0    2.0      10.0         7.0   80.0  ...   
1    3.0      27.0        26.0  146.0  ...   
2    4.0      27.0        26.0  150.0  ...   
3    5.0      29.0        29.0  150.0  ...   
4    6.0      29.0        29.0  198.0  ...   

   30_day_rolling_avg_field artillery_change  cumulative_MRL_change  \
0                                        NaN                    4.0   
1                                        NaN                    4.0   
2                                        NaN         

# 7. Standardizing and simplifying column names

### 7.1 New Column Names (5 changes)

In [18]:
# Standardizing and simplifying column names
merged_data = merged_data.rename(columns={
    'day_x': 'day_count',
    'personnel': 'personnel_count',
    'personnel*': 'personnel_estimate',
    'POW': 'prisoners_of_war',
    'day_y': 'day_of_week_number'
})

# Display the updated DataFrame
print(merged_data.head())


        date  day_count  personnel_count personnel_estimate  prisoners_of_war  \
0 2022-02-25          2             2800              about               0.0   
1 2022-02-26          3             4300              about               0.0   
2 2022-02-27          4             4500              about               0.0   
3 2022-02-28          5             5300              about               0.0   
4 2022-03-01          6             5710              about             200.0   

   daily_personnel_change  day_of_week_number  aircraft  helicopter   tank  \
0                    2800                 2.0      10.0         7.0   80.0   
1                    1500                 3.0      27.0        26.0  146.0   
2                     200                 4.0      27.0        26.0  150.0   
3                     800                 5.0      29.0        29.0  150.0   
4                     410                 6.0      29.0        29.0  198.0   

   ...  30_day_rolling_avg_field artillery_c

### 7.2 New List of Columns after changes

In [19]:
# Create a DataFrame for the columns with their respective numbers
columns_df = pd.DataFrame({
    'Column Number': range(1, len(merged_data.columns) + 1),
    'Column Name': merged_data.columns
})

# Display the DataFrame
columns_df

,Column Number,Column Name
0,1,date
1,2,day_count
2,3,personnel_count
3,4,personnel_estimate
4,5,prisoners_of_war
5,6,daily_personnel_change
6,7,day_of_week_number
7,8,aircraft
8,9,helicopter
9,10,tank


In [20]:
# Display summary statistics for the merged_data dataset
print(merged_data.describe())

                                date    day_count  personnel_count  \
count                           1110  1110.000000      1110.000000   
mean   2023-08-28 11:32:45.405405440   551.481081    316106.716216   
min              2022-02-25 00:00:00     2.000000      2800.000000   
25%              2022-11-28 06:00:00   278.250000     88020.000000   
50%              2023-08-31 12:00:00   554.500000    263255.000000   
75%              2024-05-29 18:00:00   826.750000    505970.000000   
max              2025-02-16 00:00:00  1089.000000    858390.000000   
std                              NaN   315.073706    246442.527563   

       prisoners_of_war  daily_personnel_change  day_of_week_number  \
count       1110.000000             1110.000000         1088.000000   
mean          21.581982              796.612613          540.822610   
min            0.000000                0.000000            2.000000   
25%            0.000000              470.000000          272.750000   
50%           

In [21]:
# Get the current columns of the merged_data DataFrame
current_columns = merged_data.columns

# Get the previous list of columns from columns_df
columns = columns_df['Column Name'].tolist()

# Find the missing columns by comparing with the previous list of columns
missing_columns = set(columns) - set(current_columns)

# Print the number of current columns and the missing columns
print(f"Number of current columns: {len(current_columns)}")
print(f"Missing columns: {missing_columns}")

Number of current columns: 60
Missing columns: set()


In [22]:
# Check for duplicate column names
duplicate_columns = merged_data.columns[merged_data.columns.duplicated()].unique()
print(f"Duplicate columns: {duplicate_columns}")

Duplicate columns: Index([], dtype='object')


In [23]:
# Standardize column names: lowercase and replace spaces with underscores
merged_data.columns = [col.strip().lower().replace(' ', '_') for col in merged_data.columns]

# Convert 'date' to datetime
merged_data['date'] = pd.to_datetime(merged_data['date'])

# Identify numeric columns using select_dtypes
numeric_columns = merged_data.select_dtypes(include=[np.number]).columns.tolist()
# Example output from your summary: 
# ['day_count', 'personnel_count', 'prisoners_of_war', 'daily_personnel_change', 
#  'day_of_week_number', 'sales', 'other_numeric_columns'...]

# Fill missing values in numeric columns with the mean
merged_data[numeric_columns] = merged_data[numeric_columns].fillna(merged_data[numeric_columns].mean(numeric_only=True))

# Identify non-numeric columns
non_numeric_columns = merged_data.select_dtypes(exclude=[np.number]).columns.tolist()

# Fill missing values in non-numeric columns with a default value, e.g., 'Unknown'
merged_data[non_numeric_columns] = merged_data[non_numeric_columns].fillna('Unknown')

# Remove duplicate rows
merged_data = merged_data.drop_duplicates()

# Display the cleaned DataFrame
print(merged_data.head())


        date  day_count  personnel_count personnel_estimate  prisoners_of_war  \
0 2022-02-25          2             2800              about               0.0   
1 2022-02-26          3             4300              about               0.0   
2 2022-02-27          4             4500              about               0.0   
3 2022-02-28          5             5300              about               0.0   
4 2022-03-01          6             5710              about             200.0   

   daily_personnel_change  day_of_week_number  aircraft  helicopter   tank  \
0                    2800                 2.0      10.0         7.0   80.0   
1                    1500                 3.0      27.0        26.0  146.0   
2                     200                 4.0      27.0        26.0  150.0   
3                     800                 5.0      29.0        29.0  150.0   
4                     410                 6.0      29.0        29.0  198.0   

   ...  30_day_rolling_avg_field_artillery_c

In [24]:
# Display the first 10 rows of the merged_data DataFrame
print("First 10 rows of the merged_data DataFrame:")
print(merged_data.head(10))

# Display the last 10 rows of the merged_data DataFrame
print("Last 10 rows of the merged_data DataFrame:")
print(merged_data.tail(10))

First 10 rows of the merged_data DataFrame:
        date  day_count  personnel_count personnel_estimate  prisoners_of_war  \
0 2022-02-25          2             2800              about               0.0   
1 2022-02-26          3             4300              about               0.0   
2 2022-02-27          4             4500              about               0.0   
3 2022-02-28          5             5300              about               0.0   
4 2022-03-01          6             5710              about             200.0   
5 2022-03-02          7             5840              about             200.0   
6 2022-03-03          8             9000              about             200.0   
7 2022-03-04          9             9166              about             200.0   
8 2022-03-05         10            10000              about             216.0   
9 2022-03-06         11            11000              about             232.0   

   daily_personnel_change  day_of_week_number  aircraft  helicop

In [25]:
# Define the path to save the Excel file
save_path = r'C:\Users\khaie\Desktop\2022 Russia Ukraine War\merged_data.xlsx'

# Save the merged_data DataFrame to an Excel file
merged_data.to_excel(save_path, index=False)

print(f"Data saved to {save_path} successfully.")

Data saved to C:\Users\khaie\Desktop\2022 Russia Ukraine War\merged_data.xlsx successfully.


In [26]:
# Save the merged_data DataFrame to an Excel file
merged_data.to_excel('output.xlsx', index=False)


In [27]:
# List of unique greatest losses directions
unique_directions = merged_data['greatest_losses_direction'].unique()

# Assign coordinates to each direction (example coordinates)
coordinates = {
    'North': (50.4501, 30.5234),
    'South': (46.4825, 30.7233),
    'East': (48.3794, 31.1656),
    'West': (49.9935, 36.2304),
    'Unknown': (0.0, 0.0)  # Default coordinates for unknown directions
}

# Create a DataFrame with directions and their coordinates
directions_df = pd.DataFrame({
    'Direction': unique_directions,
    'Latitude': [coordinates.get(direction, (0.0, 0.0))[0] for direction in unique_directions],
    'Longitude': [coordinates.get(direction, (0.0, 0.0))[1] for direction in unique_directions]
})

# Display the DataFrame
print(directions_df)

                         Direction  Latitude  Longitude
0                                0       0.0        0.0
1                  Sievierodonetsk       0.0        0.0
2              Kurakhove and Izyum       0.0        0.0
3           Zaporizhzhia and Izyum       0.0        0.0
4                            Izyum       0.0        0.0
..                             ...       ...        ...
61     Avdiivka, Bakhmut and Lyman       0.0        0.0
62     Bakhmut, Lyman and Avdiivka       0.0        0.0
63               Lyman and Bakhmut       0.0        0.0
64  Kupiansk, Avdiivka and Bakhmut       0.0        0.0
65                         Unknown       0.0        0.0

[66 rows x 3 columns]


In [28]:
# Get all unique greatest losses directions
unique_greatest_losses_directions = merged_data['greatest_losses_direction'].unique()

# Display the unique greatest losses directions
print("Unique Greatest Losses Directions:")
print(unique_greatest_losses_directions)

Unique Greatest Losses Directions:
[0 'Sievierodonetsk' 'Kurakhove and Izyum' 'Zaporizhzhia and Izyum'
 'Izyum' 'Izyum, Novopavlivsk' 'Popasna' 'Lyman and Kurakhove'
 'Slobozhanskyi' 'Novopavlivsk' 'Avdiivka'
 'Novopavlivsk, Kurakhove and Sievierodonetsk' 'Kurakhove'
 'Kurakhove and Avdiivka' 'Bakhmut and Zaporizhzhia'
 'Lyman and Zaporizhzhia' 'Sloviansk, Kryvyi Rih and Zaporizhzhia'
 'Bakhmut' 'Lyman' 'Avdiivka and Kryvyi Rih' 'Zaporizhzhia'
 'Kryvyi Rih and Zaporizhzhia' 'Kryvyi Rih and Bakhmut' 'Sloviansk'
 'Kharkiv and Bakhmut' 'Sievierodonetsk and Bakhmut'
 'Bakhmut and Sievierodonetsk' 'Sloviansk, Bakhmut and Kryvyi Rih'
 'Bakhmut and Avdiivka' 'Sloviansk, Bakhmut and Avdiivka'
 'Sloviansk and Bakhmut' 'Bakhmut and Kurakhove' 'Kramatorsk and Bakhmut'
 'Sloviansk and Donetsk' 'Avdiivka and Bakhmut' 'Kramatorsk' 'Mykolaiv'
 'Kramatorsk, Kryvyi Rih and Bakhmut' 'Kryvyi Rih' 'Donetsk'
 'Donetsk and Kryvyi Rih' 'Bakhmut and Kryvyi Rih' 'Bakhmut and Donetsk'
 'Kharkiv and Donetsk' 'Do

In [30]:
# Define the coordinates for each location
location_coordinates = {
    'Sievierodonetsk': (48.9481, 38.4911),
    'Kurakhove and Izyum': (48.0486, 37.2828),
    'Zaporizhzhia and Izyum': (47.8388, 35.1396),
    'Izyum': (49.2083, 37.2481),
    'Izyum, Novopavlivsk': (49.2083, 37.2481),
    'Popasna': (48.6344, 38.3778),
    'Lyman and Kurakhove': (48.9894, 37.8028),
    'Slobozhanskyi': (50.0000, 36.2500),
    'Novopavlivsk': (47.8333, 37.6667),
    'Avdiivka': (48.1408, 37.7414),
    'Novopavlivsk, Kurakhove and Sievierodonetsk': (47.8333, 37.6667),
    'Kurakhove': (47.9833, 37.2833),
    'Kurakhove and Avdiivka': (47.9833, 37.2833),
    'Bakhmut and Zaporizhzhia': (48.5882, 37.9992),
    'Lyman and Zaporizhzhia': (48.9894, 37.8028),
    'Sloviansk, Kryvyi Rih and Zaporizhzhia': (48.8667, 37.6167),
    'Bakhmut': (48.5882, 37.9992),
    'Lyman': (48.9894, 37.8028),
    'Avdiivka and Kryvyi Rih': (48.1408, 37.7414),
    'Zaporizhzhia': (47.8388, 35.1396),
    'Kryvyi Rih and Zaporizhzhia': (47.9105, 33.3918),
    'Kryvyi Rih and Bakhmut': (47.9105, 33.3918),
    'Sloviansk': (48.8667, 37.6167),
    'Kharkiv and Bakhmut': (49.9935, 36.2304),
    'Sievierodonetsk and Bakhmut': (48.9481, 38.4911),
    'Bakhmut and Sievierodonetsk': (48.5882, 37.9992),
    'Sloviansk, Bakhmut and Kryvyi Rih': (48.8667, 37.6167),
    'Bakhmut and Avdiivka': (48.5882, 37.9992),
    'Sloviansk, Bakhmut and Avdiivka': (48.8667, 37.6167),
    'Sloviansk and Bakhmut': (48.8667, 37.6167),
    'Bakhmut and Kurakhove': (48.5882, 37.9992),
    'Kramatorsk and Bakhmut': (48.7384, 37.5842),
    'Sloviansk and Donetsk': (48.8667, 37.6167),
    'Avdiivka and Bakhmut': (48.1408, 37.7414),
    'Kramatorsk': (48.7384, 37.5842),
    'Mykolaiv': (46.9750, 31.9946),
    'Kramatorsk, Kryvyi Rih and Bakhmut': (48.7384, 37.5842),
    'Kryvyi Rih': (47.9105, 33.3918),
    'Donetsk': (48.0159, 37.8028),
    'Donetsk and Kryvyi Rih': (48.0159, 37.8028),
    'Bakhmut and Kryvyi Rih': (48.5882, 37.9992),
    'Bakhmut and Donetsk': (48.5882, 37.9992),
    'Kharkiv and Donetsk': (49.9935, 36.2304),
    'Donetsk and Mykolaiv': (48.0159, 37.8028),
    'Donetsk and Kurakhove': (48.0159, 37.8028),
    'Kryvyi Rih and Donetsk': (47.9105, 33.3918),
    'Kryvyi Rih and Mykolaiv': (47.9105, 33.3918),
    'Kramatorsk and Donetsk': (48.7384, 37.5842),
    'Kramatorsk and Kryvyi Rih': (48.7384, 37.5842),
    'Kramatorsk, Avdiivka and Kryvyi Rih': (48.7384, 37.5842),
    'Donetsk, Bakhmut and Kramatorsk': (48.0159, 37.8028),
    'Kryvyi Rih and Kramatorsk': (47.9105, 33.3918),
    'Bakhmut, Avdiivka and Kramatorsk': (48.5882, 37.9992),
    'Bakhmut and Kramatorsk': (48.5882, 37.9992),
    'Donetsk and Lyman': (48.0159, 37.8028),
    'Avdiivka and Lyman': (48.1408, 37.7414),
    'Lyman and Avdiivka': (48.9894, 37.8028),
    'Lyman, Bakhmut and Avdiivka': (48.9894, 37.8028),
    'Lyman, Avdiivka and Bakhmut': (48.9894, 37.8028),
    'Bakhmut and Lyman': (48.5882, 37.9992),
    'Avdiivka, Bakhmut and Lyman': (48.1408, 37.7414),
    'Bakhmut, Lyman and Avdiivka': (48.5882, 37.9992),
    'Lyman and Bakhmut': (48.9894, 37.8028),
    'Kupiansk, Avdiivka and Bakhmut': (49.7210, 37.5986)
}

# Update the directions_df DataFrame with the coordinates
directions_df['Latitude'] = directions_df['Direction'].map(lambda x: location_coordinates.get(x, (0.0, 0.0))[0])
directions_df['Longitude'] = directions_df['Direction'].map(lambda x: location_coordinates.get(x, (0.0, 0.0))[1])

# Display the updated DataFrame
print(directions_df)

                         Direction  Latitude  Longitude
0                                0    0.0000     0.0000
1                  Sievierodonetsk   48.9481    38.4911
2              Kurakhove and Izyum   48.0486    37.2828
3           Zaporizhzhia and Izyum   47.8388    35.1396
4                            Izyum   49.2083    37.2481
..                             ...       ...        ...
61     Avdiivka, Bakhmut and Lyman   48.1408    37.7414
62     Bakhmut, Lyman and Avdiivka   48.5882    37.9992
63               Lyman and Bakhmut   48.9894    37.8028
64  Kupiansk, Avdiivka and Bakhmut   49.7210    37.5986
65                         Unknown    0.0000     0.0000

[66 rows x 3 columns]
